### Wholesale customers Data Set 
### https://archive.ics.uci.edu/ml/datasets/Wholesale+customers

Datos referentes a perfil de gastos de un supermercado lisboeta. Resumen: El conjunto de datos se refiere a los clientes de un distribuidor al por mayor. Incluye el gasto anual en unidades monetarias (m.u.) en diversas categorías de productos.
USOS: Predictor de perfil de consumo, Sistema recomendador.

Wholesale_customers_data.csv

campos: Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicatessen

En este caso la categoria podria ser cualquier de las diferentes categorias de consumo  
1) FRESH: gasto anual (m.u.) en productos frescos (continua);
2) leche: el gasto anual (m.u.) en los productos lácteos (continua);
3) comestibles: el gasto anual (m.u.) en los productos comestibles (continua);
4) Congelado: gasto anual (m.u.) en productos congelados (continua)
5) DETERGENTS_PAPER: gasto anual (m.u.) en detergentes y productos de papel (continua)
6) Delicatessen: gasto anual (m.u.) y en productos delicatessen (continuos);
7) Canal: Canal customersâ € ™ - Horeca (Hotel / Restaurante / Café ©) o canal de venta al por menor (nominal)
8) REGIÓN: customersâ € ™ â € Región "Lisnon, Oporto o Otros (nominal)


Elegimos: delicatessen como categoria, ya que el supermercado está interesado en categorizar los consumidores de este tipo.
¿con que tipo de comida se correlaciona más?
Sistema de recomendación: ¿que se debería sugerir como compra complementaria a un ciente?


In [1]:
import numpy as np
# from sklearn import preprocessing, cross_validation, neighbors
import pandas as pd

dataset_name = "Wholesale_customers_data"
format = pd.read_csv(dataset_name + '-format.csv')

df = pd.read_csv(dataset_name + '.csv')
df['Class'] = (df['Delicatessen'] > 2000).astype(int)
df = df.drop('Delicatessen',axis=1)
labels = {'Usual Consumer': 0, 'Great Consumer': 1}

fmts = []
for c, d in {n: {format['field'][i]: float(vv) for i, vv in list(enumerate(v))}
             for n, v in dict(format.drop('field', axis=1).drop('Delicatessen', axis=1)).items()}.items():
    d.update(dict(title=c))
    fmts.append(d)

field_names = [field for field in df.drop(['Class'], 1)]

In [2]:
X = np.array(df.drop(['Class'], 1), dtype=float)
y = np.array(df['Class'], dtype=float)

In [3]:
from sklearn.preprocessing import StandardScaler
scalerX = StandardScaler()
scalery = StandardScaler()
Xn = np.apply_along_axis(scalerX.fit_transform,0,X)
yn = np.apply_along_axis(scalery.fit_transform,0,y)

In [4]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2).fit(X)
Xt = pca.transform(X)
xx1 = Xt[y == labels['Usual Consumer']][:, 0]
yy1 = Xt[y == labels['Usual Consumer']][:, 1]

xx2 = Xt[y == labels['Great Consumer']][:, 0]
yy2 = Xt[y == labels['Great Consumer']][:, 1]

In [6]:
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans

usual_ones = Xt[y == labels['Usual Consumer']]
great_ones = Xt[y == labels['Great Consumer']]

c_min = 2
c_max = 20
metric = 'euclidean'

silh = []
km_model = [0] * c_max
for t in range(2, 20):
    km_model[t - c_min] = KMeans(n_clusters=t).fit(great_ones)
    silh.append(silhouette_score(great_ones, km_model[t - c_min].labels_, metric=metric))

idx_great = np.argmax(silh)
n_clusters_great = idx_great + c_min
cluster_lbl_great = km_model[idx_great].labels_
centroid_great = {}
for each in np.unique(cluster_lbl_great).tolist():
    centroid_great[each] = np.mean(great_ones[cluster_lbl_great == each], 0)
    
silh = []
km_model = [0] * c_max
for t in range(2, 20):
    km_model[t - c_min] = KMeans(n_clusters=t).fit(usual_ones)
    silh.append(silhouette_score(usual_ones, km_model[t - c_min].labels_, metric=metric))

idx_usual = np.argmax(silh)
n_clusters_usual = idx_usual + c_min
cluster_lbl_usual = km_model[idx_usual].labels_
centroid_usual = {}
for each in np.unique(cluster_lbl_usual).tolist():
    centroid_usual[each] = np.mean(usual_ones[cluster_lbl_usual == each], 0)

In [12]:
pca.__dict__

{'components_': array([[  4.85437338e-06,  -3.29602341e-06,  -9.81846574e-01,
          -1.05527899e-01,  -4.08588147e-02,  -1.51331852e-01,
           1.64436935e-02],
        [ -2.42258694e-05,  -6.59431479e-07,   7.91577778e-02,
          -5.18815161e-01,  -7.68137246e-01,   1.57810130e-02,
          -3.66443726e-01]]),
 'copy': True,
 'explained_variance_': array([  1.63906785e+08,   1.44654419e+08]),
 'explained_variance_ratio_': array([ 0.46798756,  0.4130181 ]),
 'mean_': array([  1.32272727e+00,   2.54318182e+00,   1.20002977e+04,
          5.79626591e+03,   7.95127727e+03,   3.07193182e+03,
          2.88149318e+03]),
 'n_components': 2,
 'n_components_': 2,
 'n_samples_': 440L,
 'noise_variance_': 8335255.5136779323,
 'whiten': False}

In [10]:
centroid_usual, centroid_great

({0: array([ 5945.95571666,  4444.14677679]),
  1: array([-14014.14220861,   5894.86775211]),
  2: array([  5950.77586828, -16573.00237945])},
 {0: array([-4906.72430097,  -963.14229213]),
  1: array([-24643.91878405, -43214.70961087])})

### VARIOS CLASIFICADORES CON CROSSFOLDER 

In [7]:
# -VARIOS CLASIFICADORES SVC, RF, KNN, LOG_REG ----------------------------

from sklearn.cross_validation import KFold

#  definimos la funcion de cross validation
def run_cv(X,y,clf_class,**kwargs):
    # Construimos el objeto kfolds 
    kf = KFold(len(y),n_folds=5,shuffle=True)
    y_pred = y.copy()
    
    # tenemos  que iterar sobre todos los kfolds
    for train_index, test_index in kf:
        X_train, X_test = X[train_index], X[test_index]
        y_train = y[train_index]
        
        
        # Iniciamos el clasificador con los keywords arguments...
        clf = clf_class(**kwargs)
        clf.fit(X_train,y_train)
        y_pred[test_index] = clf.predict(X_test)
    return y_pred

In [12]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.linear_model import LogisticRegression as LOG_REG

from sklearn.metrics import accuracy_score as accuracy
from sklearn.metrics import confusion_matrix
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler


# ------- CONFIANZA, RECALL, PRECISION, ETC. ---------------
print("--------  Confianzas de los calsificadores: \n \n ")
pred_SVC = run_cv(X,y,SVC)
print("Confianza de SVC = ", accuracy(y, pred_SVC))
print("MATRIZ DE CONFUSION de SVC \n", confusion_matrix(y, pred_SVC), "\n")

pred_KNN = run_cv(X,y,KNN)
print("Confianza de KNN = ", accuracy(y, pred_KNN))
print("MATRIZ DE CONFUSION de KNN \n", confusion_matrix(y, pred_KNN), "\n")

pred_LOG_REG = run_cv(X,y,LOG_REG)
print("Confianza de  Log. Regression = ", accuracy(y, pred_LOG_REG))
print("MATRIZ DE CONFUSION de Log. Regression \n", confusion_matrix(y, pred_LOG_REG), "\n")

#print "%.3f" % accuracy(y, run_cv(X,y,RF))
pred_RF = run_cv(X,y,RF)
print("Confianza de Random forest:", accuracy(y, pred_RF))
print("MATRIZ DE CONFUSION de RANDOM FOREST \n", confusion_matrix(y, pred_RF), "\n")

--------  Confianzas de los calsificadores: 
 
 
Confianza de SVC =  0.781818181818
MATRIZ DE CONFUSION de SVC 
 [[344   0]
 [ 96   0]] 

Confianza de KNN =  0.754545454545
MATRIZ DE CONFUSION de KNN 
 [[309  35]
 [ 73  23]] 

Confianza de  Log. Regression =  0.809090909091
MATRIZ DE CONFUSION de Log. Regression 
 [[337   7]
 [ 77  19]] 

Confianza de Random forest: 0.745454545455
MATRIZ DE CONFUSION de RANDOM FOREST 
 [[307  37]
 [ 75  21]] 



### Calculamos los limites de los SLIDES de BOKEH para que estén derentro del margen de los valores del data set

In [14]:
print(np.amin(X,axis=0))
print(np.amax(X,axis=0))

[ 1  1  3 55  3 25  3]
[     2      3 112151  73498  92780  60869  40827]


In [ ]:
# RECORREMOS AHORA TODO EL DATAFRAME DE FORMA QUE SEA APLICABLE A CUALQUIERA DE LOS DATA FRAMES DE LOS CASOS... 

lista_minimos_col = []
lista_maximos_col = []

# recorremos las columnas del DATA FRAME
for i in range (df.shape[1]-1):                  # EL -1 es porque exxluyo el ultimo campo de la categoria o label 
    #print df.iloc[1][i:1]
    #print df.values[1][i]
    # añadimos el MIN y MAXde esa columna del dataframe
    lista_minimos_col.append(df.ix[df[df.columns[i]].idxmin()][df.columns[i]])
    lista_maximos_col.append(df.ix[df[df.columns[i]].idxmax()][df.columns[i]])

print lista_minimos_col
print lista_maximos_col

In [ ]:
df.columns[4:]

In [ ]:
# graficas scattered tomadas de https://www.mapr.com/blog/predicting-airbnb-listing-prices-scikit-learn-and-apache-spark  
import matplotlib.pyplot as plt
#scattercols = ['price','accommodates', 'number_of_reviews', 'reviews_per_month', 'beds', 'availability_30', 'review_scores_rating']
scattercols = df.columns[7:]
axs = pd.scatter_matrix(df[scattercols],
                        figsize=(12, 12), c='red')

plt.figure(figsize=(1920,1080))
plt.show()

# Ahora calculamos los puntos cercanos a un punto dado para calcular donde están mas proximas los puntos de categoria diferente, (cancerosas). Esto nos ayudará a seleccionar las direcciones más relevantes a la hora de estudiar la evolución del punto en cuestión.
##UTLIZAMOS UN EJEMPLO SENCILLO DE DATAFRAME EN SU LUGAR

In [ ]:
print X[y==2].shape
print X[y==4].shape

In [ ]:
classes = {}
for each in np.unique(y).tolist():
    classes[each] = X[y==each]

In [ ]:
classes = {}
for each in np.unique(y).tolist():
    classes[each] = X[y==each]

benign = classes[labels['benign']]
malignant = classes[labels['malignant']]

b_dims = benign.shape
m_dims = malignant.shape

benign = np.reshape(benign, b_dims + (1,))
malignant = np.reshape(malignant, m_dims + (1,))

np.tile(benign, [1, 1, m_dims[0]]) - 

# PREDICCIONES CON INPUTS

In [ ]:
# vamos a hacer predicciones :

example_measures = np.array([[10,5,5,3,6,7,7,10,1]])
example_measures = example_measures.reshape(len(example_measures), -1)
prediction = clf.predict(example_measures)
print(prediction)

In [ ]:
print(df.values)

# 2 PARTE: gráficas-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


In [ ]:
# condiciones que definen nuevos datasets
df_tumoral = df[df.Class > 3]
df_NO_tumoral = df[df.Class < 3]

df_tumoral.head()

# graficas scattered tomadas de https://www.mapr.com/blog/predicting-airbnb-listing-prices-scikit-learn-and-apache-spark  
import matplotlib.pyplot as plt
#scattercols = ['price','accommodates', 'number_of_reviews', 'reviews_per_month', 'beds', 'availability_30', 'review_scores_rating']
scattercols = df.columns
axs = pd.scatter_matrix(df[scattercols],
                        figsize=(12, 12), c='red')

plt.figure(figsize=(1920,1080))
plt.show()

In [ ]:
# definimos una funcion distancia de un registro del data se

In [ ]:
df_NO_tumoral.head()

In [ ]:
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from matplotlib import style

style.use('ggplot')

fig = plt.figure()
ax1 = fig.add_subplot(111, projection='3d')

x = df_tumoral.Clump_Thickness
y = df_tumoral.Uniform_Cell_Shape
z = df_tumoral.Unifom_Cell_Size

x2 = df_NO_tumoral.Clump_Thickness
y2 = df_NO_tumoral.Uniform_Cell_Shape
z2 = df_NO_tumoral.Unifom_Cell_Size


ax1.scatter(x, y, z, c='g', marker='o')
ax1.scatter(x2, y2, z2, c ='r', marker='o')

ax1.set_xlabel('Espesor')
ax1.set_ylabel('Forma Celular')
ax1.set_zlabel(u'Tamaño Celular')

plt.show()

In [ ]:
#imprimimos las 3 primeras variables de prueba

from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from matplotlib import style

style.use('ggplot')

fig = plt.figure()
ax1 = fig.add_subplot(111, projection='3d')



x = df_tumoral.eval(df_tumoral.columns[1])
y =  df_tumoral.eval(df_tumoral.columns[2])
z =  df_tumoral.eval(df_tumoral.columns[3])

x2 = df_NO_tumoral.eval(df_NO_tumoral.columns[1])
y2 = df_NO_tumoral.eval(df_NO_tumoral.columns[2])
z2 = df_NO_tumoral.eval(df_NO_tumoral.columns[3])


ax1.scatter(x, y, z, c='g', marker='o')
ax1.scatter(x2, y2, z2, c ='r', marker='o')

ax1.set_xlabel('Espesor')
ax1.set_ylabel('Forma Celular')
ax1.set_zlabel(u'Tamaño Celular')

plt.show()

In [ ]:
print(df.columns)

In [ ]:
print(df.columns[9])
print(len(df.columns))
      

In [ ]:
# funcion GENERICA QUE DIBUJA UNA COMBINACION particulr de las variable

from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from matplotlib import style

def dibuja_grafica(i,j,k):

    style.use('ggplot')

    fig = plt.figure()
    ax1 = fig.add_subplot(111, projection='3d')


    x = df_tumoral.eval(df_tumoral.columns[i])
    y =  df_tumoral.eval(df_tumoral.columns[j])
    z =  df_tumoral.eval(df_tumoral.columns[k])

    x2 = df_NO_tumoral.eval(df_NO_tumoral.columns[i])
    y2 = df_NO_tumoral.eval(df_NO_tumoral.columns[j])
    z2 = df_NO_tumoral.eval(df_NO_tumoral.columns[k])

    ax1.scatter(x, y, z, c='g', label='TUMORALES', marker='o')
    ax1.scatter(x2, y2, z2, c ='r', label='SANAS', marker='o')
    
    #plt.scatter(x,y, label='TUMORALES', color='g', s=25, marker="o")
    #plt.scatter(x2,y2, label='SANAS', color='r', s=25, marker="o")
 
    ax1.set_xlabel(df_tumoral.columns[i])
    ax1.set_ylabel(df_tumoral.columns[j])
    ax1.set_zlabel(df_tumoral.columns[k])
    
    plt.title('Breast Cancer Dataset Analysis               ')
    plt.legend()

    plt.show()  
    #plt.savefig('graf_'+df_tumoral.columns[i]+df_tumoral.columns[j]+df_tumoral.columns[k]+'.png')

dibuja_grafica(1,2,4)
    
    

    

In [ ]:
df.columns[-1]

In [ ]:
# representacion de todas las GRAFICAS combinacion total de variables

for counter1 in range(0,len(df.columns)-4):
    for counter2 in range(counter1 + 1,len(df.columns)):
        for counter3 in range(counter2 + 1,len(df.columns)):
            try:
                dibuja_grafica(counter1,counter2,counter3)
                #print counter1,",", counter2,",", counter3
            except Exception:
                pass

In [ ]:
# recordatorio de graficas en 2D

import matplotlib.pyplot as plt

x = [1,2,3,4,5,6,7,8]
y = [5,2,4,2,1,4,5,2]

x2 = [2,3,3,7,5,2,5]
y2 = [3,1,0,2,4,5,2]


#style.use('ggplot')

fig = plt.figure() 

# AÑADIMOS UN SUBPLOT sobre fig
ax1 = fig.add_subplot(111)

ax1.scatter(x, y, c='g', marker='o')
ax1.scatter(x2, y2, c ='r', marker='o')

plt.scatter(x,y, label='TUMORALES', color='g', s=25, marker="o")
plt.scatter(x2,y2, label='NORMALES', color='r', s=25, marker="o")

plt.xlabel('x')
plt.ylabel('y')
plt.title('Breast Cancer Analysis')
plt.legend()
plt.show()

In [ ]:
# GRAFICA EN 2D  con la clase como Z
import matplotlib.pyplot as plt
from matplotlib import style

def dibuja_grafica2D(i,j,k):

    style.use('ggplot')
    
    x = df_tumoral.eval(df_tumoral.columns[i])
    y =  df_tumoral.eval(df_tumoral.columns[j])

    x2 = df_NO_tumoral.eval(df_NO_tumoral.columns[i])
    y2 = df_NO_tumoral.eval(df_NO_tumoral.columns[j])
    
    fig = plt.figure() 

    # AÑADIMOS UN SUBPLOT sobre fig
    ax1 = fig.add_subplot(111)

    ax1.scatter(x, y, c='g', marker='o')
    ax1.scatter(x2, y2, c ='r', marker='o')

    plt.scatter(x,y, label='TUMORALES', color='g', s=25, marker="o")
    plt.scatter(x2,y2, label='SANAS', color='r', s=25, marker="o")

    # ax1.scatter(x, y, z, c='g', marker='o')
    # ax1.scatter(x2, y2, z2, c ='r', marker='o')

    ax1.set_xlabel(df_tumoral.columns[i])
    ax1.set_ylabel(df_tumoral.columns[j])


    plt.title('Breast Cancer Dataset Analysis - Human Forecast')
    plt.legend()
    #plt.show()
    plt.savefig('graficos/graf_'+df_tumoral.columns[i]+df_tumoral.columns[j]+'.png')


dibuja_grafica2D(1,2,9)

In [ ]:
# pero ahora solo estamos interesados en combinar dos a dos 
# con z que sea la clase de tumor que es el indice 9

for counter1 in range(0,len(df.columns)-2):
    for counter2 in range(counter1 + 1,len(df.columns)-1):
            try:
                dibuja_grafica2D(counter1,counter2,9)
                #print counter1,",", counter2
            except Exception:
                pass